## **WRF WORK ASSIGMENT #1** 

In [590]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import integrate
import cmocean as cmo
import xarray as xr
import imageio
import os
plt.style.use('seaborn')

<h2><strong>Parameters</strong></h2>

In [252]:
wind_speed = 7*1e-3
wind_direc = 140
dx = dy = 18
time_step  = 0.04
uwind = np.cos(np.deg2rad(wind_direc))*wind_speed
vwind = np.sin(np.deg2rad(wind_direc))*wind_speed

In [189]:
T = np.array([[21.2, 22.8, 23.5, 24.8, 26.5], 
              [22.7, 24.4, 25.2, 26.5, 28.4], 
              [24.3, 26.1, 27.0, 28.4, 30.4], 
              [26.0, 27.9, 28.9, 30.4, 32.5],
              [27.8, 29.9, 30.9, 32.5, 34.8],
              [29.7, 32.0, 33.0, 34.8, 37.2], 
              [31.8, 34.2, 35.3, 37.2, 39.8]])

#### **Time step**

<h3>$\frac{C*\Delta t}{\Delta r} \le 0.06$</h3>

In [182]:
tstep = lambda ws, wd, dx, dy: round(0.06*dx/ws*abs((np.cos(np.deg2rad(wd)))) + 
                                     0.06*dy/ws*abs(np.sin(np.deg2rad(wd))),2)
time_step = tstep(wind_speed, wind_direc, dx, dy)
print(f'Our time step must be less than {time_step} seconds')

Our time step must be less than 216.0 seconds


#### **Choosing** $\Delta t = 200s$

In [318]:
time_step = 200

In [488]:
N = round(3600/200)+1
print(f'Total N simulation is {N}')

Total N simulation is 19


<h2><center><strong>Total derivative for Temperature</strong></center></h2>

<h3><strong>$$\frac{\partial T}{\partial t} = u\frac{\partial T}{\partial x} + v\frac{\partial T}{\partial y}$$</strong></h3>

In [587]:
DTdt = np.array([T]*N)    

In [602]:
def eulerm(f,dx,dy,dt,u,v):
    dTdx,dTdy = np.gradient(f.T, dx,dy)
    dTdx = dTdx.T
    dTdy = -dTdy.T
    F = f+dt*(u*dTdx+v*dTdy)
    return F

In [603]:
for n in range(N-1):
    DTdt[n+1,:,:] = eulerm(DTdt[n,:,:], dx,dy,time_step,uwind,vwind)

In [604]:
DTdt.shape

(18, 7, 5)

In [622]:
def xrgradient(DTdt,dx,dy):
    dtdx = [np.gradient(DTdt[n,:,:],dx,dy) for n in range(DTdt.shape[0])]
    print(len(dtdx[0]))
    """
    ds = xr.DataArray(dtdx, dims=['time','x', 'y'], 
                      coords={'time':np.arange(18),'y':np.arange(7),'x':np.arange(5)})
    """

In [623]:
xrgradient(DTdt,dx,dy)

2


In [554]:
%%capture
def temp_advect_plot(n,N):
    fig, ax = plt.subplots(figsize=(10,6), dpi=80)
    pls = ax.contourf(DTdt[n,:,:], levels=np.linspace(15,41,48, endpoint=True), cmap=cmo.cm.delta, 
                      zorder=1, vmin=20, vmax=40)
    CS  = ax.contour(DTdt[n,:,:], levels=12, zorder=2, linewidths=0.6, colors='black', vmin=20, vmax=40)
    ax.clabel(CS, CS.levels, inline=True, fontsize=8, fmt='%1.1f', colors='black')
    ax.grid(linestyle='--',color='gray',alpha=0.5)
    ax.set(xlabel='X axis', ylabel='Y axis') 
    ax.set_title(f'Time={N} seconds', loc='left')
    ax.set_xticks(np.arange(0,5))
    colorbar_ax = fig.add_axes([0.91, 0.125, 0.035, 0.755])
    fig.colorbar(pls, cax=colorbar_ax, aspect=15, label='Temperature (ºC)', ticks=np.arange(15,41,3))
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    return image
kwargs_write = {'fps':1.0, 'quantizer':'nq'}
imageio.mimsave('./Temp_advect_HW1_CEO.gif', 
                [temp_advect_plot(n, N) for n,N in enumerate(np.arange(0,3600+1,time_step))], fps=1)